In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.distributions import Categorical
import gym

In [3]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    
    def clear_memory(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, n_latent_var):
        super(ActorCritic, self).__init__()

        # actor
        self.action_layer = nn.Sequential(
                nn.Linear(state_dim, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, action_dim),
                nn.Softmax(dim=-1)
                )
        
        # critic
        self.value_layer = nn.Sequential(
                nn.Linear(state_dim, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, 1)
                )
        
    def forward(self):
        raise NotImplementedError
        
    def act(self, state, memory):
        state = torch.from_numpy(state).float().to(device) 
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        action = dist.sample()
        ###
        log_probs = dist.log_prob(action)
        ###
        memory.states.append(state)
        memory.actions.append(action)
        memory.logprobs.append(dist.log_prob(action))
        
        return action.item(), log_probs
    
    def evaluate(self, state, action):
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        
        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        
        state_value = self.value_layer(state)
        
        return action_logprobs, torch.squeeze(state_value), dist_entropy
        
class PPO:
    def __init__(self, state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip):
        self.lr = lr
        self.betas = betas
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.policy = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr, betas=betas)
        self.policy_old = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()
    
    def update(self, memory):   
        # Monte Carlo estimate of state rewards:
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
    
        #for reward in reversed(memory.rewards):
       #     discounted_reward = reward + (self.gamma * discounted_reward)
       #     rewards.insert(0,discounted_reward)
        # Normalizing the rewards:
        rewards = torch.FloatTensor(rewards).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)
        
        # convert list to tensor
        old_states = torch.stack(memory.states).to(device).detach()
        old_actions = torch.stack(memory.actions).to(device).detach()
        old_logprobs = torch.stack(memory.logprobs).to(device).detach()
        self.os = old_states
        # Optimize policy for K epochs:
        for _ in range(self.K_epochs):
            # Evaluating old actions and values :
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)
            
            # Finding the ratio (pi_theta / pi_theta__old):
            ratios = torch.exp(logprobs - old_logprobs.detach())
                
            # Finding Surrogate Loss:
            advantages = rewards - state_values.detach()
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            self.loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            self.ratios, self.log_probs, self.advantages = ratios, log_probs, advantages

            # take gradient step
            self.optimizer.zero_grad()
            self.loss.mean().backward()
            self.optimizer.step()
        
        # Copy new weights into old policy:
        self.policy_old.load_state_dict(self.policy.state_dict())



In [8]:
#env = gym.make('MountainCar-v0')
#env = gym.make('LunarLander-v2')
env = gym.make('CartPole-v0')
#buffer = ReplayPool(100, env.observation_space.shape[0], env.action_space.n)
############## Hyperparameters ##############
#env_name = "MountainCar-v0"
# creating environment
#env = gym.make('MountainCar-v0')
#env = env1
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
render = False
solved_reward = 190         # stop training if avg_reward > solved_reward
log_interval = 20           # print avg reward in the interval
max_episodes = 50000        # max training episodes
max_timesteps = 2000         # max timesteps in one episode
n_latent_var = 64           # number of variables in hidden layer
update_timestep = 2000      # update policy every n timesteps
lr = 0.0005
betas = (0.9, 0.999)
gamma = 0.99                # discount factor
K_epochs = 10                # update policy for K epochs
eps_clip = 0.2              # clip parameter for PPO
random_seed = None
render = True
#############################################

if random_seed:
    torch.manual_seed(random_seed)
    env.seed(random_seed)

memory = Memory()
ppo = PPO(state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip)
print(lr,betas)

# logging variables
running_reward = 0
avg_length = 0
timestep = 0

# training loop
for i_episode in range(1, max_episodes+1):
    state = env.reset()
    for t in range(max_timesteps):
        timestep += 1

        # Running policy_old:
        action,log_probs = ppo.policy_old.act(state, memory)
        next_state, reward, done, _ = env.step(action)
    
        # Saving reward and is_terminal:
        memory.rewards.append(reward)
        memory.is_terminals.append(done)
        state = next_state
        # update if its time
        if timestep % update_timestep == 0:
            ppo.update(memory)
            memory.clear_memory()
            timestep = 0

        running_reward += reward
        if done:
            break

    avg_length += t

    # stop training if avg_reward > solved_reward
    if running_reward > (log_interval*solved_reward):
        env.render()
        break

    # logging
    if i_episode % log_interval == 0:
        avg_length = int(avg_length/log_interval)
        running_reward = int((running_reward/log_interval))

        print('Episode {} \t avg length: {} \t reward: {}'.format(i_episode, avg_length, running_reward))
        running_reward = 0
        avg_length = 0

0.0005 (0.9, 0.999)
Episode 20 	 avg length: 15 	 reward: 16
Episode 40 	 avg length: 16 	 reward: 17
Episode 60 	 avg length: 21 	 reward: 22
Episode 80 	 avg length: 21 	 reward: 22
Episode 100 	 avg length: 15 	 reward: 16
Episode 120 	 avg length: 28 	 reward: 29
Episode 140 	 avg length: 19 	 reward: 20
Episode 160 	 avg length: 23 	 reward: 24
Episode 180 	 avg length: 22 	 reward: 23
Episode 200 	 avg length: 25 	 reward: 26
Episode 220 	 avg length: 30 	 reward: 31
Episode 240 	 avg length: 23 	 reward: 24
Episode 260 	 avg length: 37 	 reward: 38
Episode 280 	 avg length: 31 	 reward: 32
Episode 300 	 avg length: 40 	 reward: 41
Episode 320 	 avg length: 39 	 reward: 40
Episode 340 	 avg length: 62 	 reward: 63
Episode 360 	 avg length: 35 	 reward: 36
Episode 380 	 avg length: 51 	 reward: 52
Episode 400 	 avg length: 64 	 reward: 65
Episode 420 	 avg length: 89 	 reward: 90
Episode 440 	 avg length: 102 	 reward: 103
Episode 460 	 avg length: 99 	 reward: 100
Episode 480 	 a

In [9]:
env.close()